In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate, KFold, StratifiedKFold, GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

import pandas as pd
import numpy as np

from datetime import datetime

In [2]:
def resultados_aux(resultados):
    """Exibe os resultados de acurácia no formato intervalo de confiança após rodar o cross validate
    
    Args:
        resultados(array): contém as acurácias para cada configuração do dataset
    """
    media = resultados.mean()
    dp = resultados.std()
    print(f'Acurácia de [{round((media-2*dp)*100, 2)}%, {round((media+2*dp)*100, 2)}%].')
    
    pass


In [3]:
# Usando os mesmos dados do curso 1 (Machine learning: introdução a classificação com SKLearn

uri = 'https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv'
dados = pd.read_csv(uri)

# Fazer algumas modificações no conjunto de dados:
#    - Renomear para português; milhas -> km; ano do modelo -> idade do modelo; (yes, no) -> (1, 0)
nomes = {
    'mileage_per_year' : 'milhas_por_ano',
    'model_year' : 'ano_do_modelo',
    'price' : 'preco',
    'sold' : 'vendido'
}

vendido_map = {
    'yes' : 1,
    'no' : 0
}

dados = dados.rename(columns=nomes)
dados['km_por_ano'] = dados.milhas_por_ano*1.609344
dados['idade_do_modelo'] = datetime.today().year - dados.ano_do_modelo
dados.vendido = dados.vendido.map(vendido_map)

dados = dados.drop(columns=['Unnamed: 0', 'milhas_por_ano', 'ano_do_modelo'])
display(dados.head())

# Separar entre features e labels
x = dados[['preco', 'km_por_ano', 'idade_do_modelo']]
y = dados['vendido']

,preco,vendido,km_por_ano,idade_do_modelo
0,30941.02,1,35085.308544,22
1,40557.96,1,12622.084992,24
2,89627.50,0,11440.826496,16
3,95276.14,0,43167.434112,7
4,117384.68,1,12770.144640,8


# 01. Validação cruzada e aleatoriedade inicial

In [4]:
# Instanciando o modelo
modelo = DecisionTreeClassifier(max_depth=2)

# Aplicando o cross validate
resultados = cross_validate(modelo, x, y, cv=5)['test_score']

# Exibição considerando intervalo de confiança 95%
resultados_aux(resultados)

Acurácia de [75.21%, 76.35%].


# 02. KFold e aleatoriedade

A forma de fazer com que a validação cruzada rode com o conjunto de dados previamente embaralhada é por meio de um gerador de validação cruzada que, no caso, é o KFold

In [5]:
np.random.seed(333)

modelo = DecisionTreeClassifier(max_depth=2)
cv = KFold(n_splits=5, shuffle=True)
resultados = cross_validate(modelo, x, y, cv=cv)['test_score']

resultados_aux(resultados)

Acurácia de [74.25%, 77.31%].


# 03. Estratificação

In [6]:
# Simulando uma situação de desbalanceamento dos labels enviados ao cross validate ou de 'azar':

# Ordenar o dataset pelos labels
dados_azar = dados.sort_values(by='vendido', ascending=True)
x_azar = dados_azar.drop(columns=['vendido'])
y_azar = dados_azar['vendido']

# Enviar ao cross validate sem utilzar o shuffle
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x_azar, y_azar, cv=KFold(n_splits=5))['test_score']
resultados_aux(resultados)

Acurácia de [32.22%, 87.18%].


In [7]:
# Usando o shuffle, a acurácia volta a um nível normal
np.random.seed(333)
resultados_shuffle = cross_validate(modelo, x_azar, y_azar, 
                                    cv=KFold(n_splits=5, shuffle=True))['test_score']
resultados_aux(resultados_shuffle)

Acurácia de [73.88%, 77.68%].


In [8]:
# Também é possível fazer com que teste e treino mantenham as proporções do dataset, usando stratifiedkfold
np.random.seed(333)
resultados_shuffle = cross_validate(modelo, x_azar, y_azar, 
                                    cv=StratifiedKFold(n_splits=5, shuffle=True))['test_score']
resultados_aux(resultados_shuffle)

Acurácia de [73.37%, 78.19%].


# 04. Dados agrupáveis

Num caso em que se tem várias entradas que podem ser classificadas em um mesmo grupo real, é desejado que o algoritmo não use essas entradas similares separadas entre treino e teste: somente em um cenário ou outro.

Foi usado como exemplo em aula um algortimo que usa entradas de pacientes, sendo assim seria desejável agrupar todas as entradas de mesmos pacientes para que o algortimo só fosse testado em pacientes novos e não acabasse testado nos mesmos em que foi treinado.

O cross_validate suporta esse aspecto com o parâmetro groups, alinhado ao GroupKFold, e aqui será usada uma coluna aleatória de 'modelo do carro'. Vale destacar que essa coluna nova não é usada para treinar o modelo, apenas para agrupá-lo na divisão

In [9]:
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=dados.shape[0])
dados.modelo.unique()

array([24, 22, 16,  8,  9, 14, 21, 12, 20,  5, 15, 17, 19, 18,  7, 11, 13,
       23, 10, 25, 26,  4,  6,  3], dtype=int64)

In [10]:
np.random.seed(333)
modelo = DecisionTreeClassifier(max_depth=2)
cv = GroupKFold(n_splits=5)
resultados = cross_validate(modelo, x, y, groups=dados.modelo, cv=cv)['test_score']
resultados_aux(resultados)

Acurácia de [72.29%, 79.25%].


# 05. Pipeline de treino e validação 

Supondo que os dados precisassem passar por uma normalização, por exemplo, antes do treinamento do modelo, isso deveria ser feito após a divisão entre amostras de treino e teste uma vez que o próprio scaler é ajustado à amostra fornecida (no caso, a de treino). Quando usa-se o cross_validate, a intenção é justamente pela intenção de realizar um rodízio nessa divisão treino/teste, então o scaler deveria ser aplicado a cada iteração com uma diferente divisão. Para tornar isso possível, existe a pipeline implementada no sklearn, por meio das quais as operações podem ser passadas com uma dada ordem.

In [11]:
np.random.seed(333)

# Criando o objeto pipeline
scaler = StandardScaler()
modelo = SVC()
pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

# Fazendo o cross validate: ao invés de ter como input o modelo, tem-se a pipeline
resultados = cross_validate(pipeline, x, y, cv=KFold(n_splits=5, shuffle=True))['test_score']
resultados_aux(resultados)

Acurácia de [75.89%, 77.31%].
